Object Detection API Demo

https://colab.research.google.com/github/MarkDaoust/models/blob/tf_slim-2/research/object_detection/object_detection_tutorial.ipynb

Run in Google Colab	View source on GitHub
Welcome to the Object Detection API. This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image.

Important: This tutorial is to help you through the first step towards using Object Detection API to build models. If you just just need an off the shelf model that does the job, see the TFHub object detection example.

Setup
Important: If you're running on a local machine, be sure to follow the installation instructions. This notebook includes only what's necessary to run in Colab.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#
# Define Variables
#
project_folder = '/content/drive/Shareddrives/HW_SHARE/colab08'
train_dir = project_folder + '/training'
tflite_model_dir = project_folder + '/tflite_model'
saved_model_dir = tflite_model_dir + '/saved_model'
pipeline_file = project_folder + '/models/mymodel/pipeline_file.config'


In [ ]:
#
# Install
#
!mkdir {project_folder}
!cd {project_folder}
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim



In [ ]:
#
# Get tensorflow/models or cd to parent directory of the repository.
#
#!cd {project_folder}
!git clone --depth 1 https://github.com/tensorflow/models {project_folder}/models

Cloning into '/content/drive/MyDrive/colab08/models'...
remote: Enumerating objects: 4055, done.
remote: Counting objects: 100% (4055/4055), done.
remote: Compressing objects: 100% (3085/3085), done.
remote: Total 4055 (delta 1184), reused 2788 (delta 913), pack-reused 0
Receiving objects: 100% (4055/4055), 54.69 MiB | 13.20 MiB/s, done.
Resolving deltas: 100% (1184/1184), done.
Updating files: 100% (3668/3668), done.


In [ ]:
%cd {project_folder}/models/research
!pwd
!protoc object_detection/protos/*.proto --python_out=.

/content/drive/MyDrive/colab08/models/research
/content/drive/MyDrive/colab08/models/research


In [ ]:

%cd {project_folder}/models/research
!pwd
!cp object_detection/packages/tf2/setup.py .
!pip install . --upgrade

/content/drive/MyDrive/colab08/models/research
/content/drive/MyDrive/colab08/models/research
Processing /content/drive/MyDrive/colab08/models/research
  Preparing metadata (setup.py) ... done
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1697354 sha256=772bef99e8e2921b1e4873d25fafdf11f40f4d05355373a1041096bfd242084a
  Stored in directory: /tmp/pip-ephem-wheel-cache-3mj6van_/wheels/01/8d/82/329c33195c6a7089dad1664f027e7a3f575fa4364d4b28a326
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [ ]:
#
# Imports
#
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [ ]:
#
# Import the object detection module.
#
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
#
# Patches
#
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

Convert Model to TensorFlow Lite

In [ ]:
!mkdir {tflite_model_dir}
output_directory = tflite_model_dir

last_model_path = train_dir

!python {project_folder}/models/research/object_detection/export_tflite_graph_tf2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --max_detections 100 \
    --pipeline_config_path {pipeline_file}


2023-11-21 12:36:56.783917: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-21 12:36:56.783990: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-21 12:36:56.785749: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-21 12:36:58.815314: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
I1121 12:37:11.608186 137329451315200 api.py:460] feature_map_spatial_dims: [(19, 19), (10, 10), (5, 5), (3, 3), (2, 2), (1, 1)]
INFO:tensorflow:depth of additional conv before box predictor: 0
I1121 12:37:13.689518 137329451315200 convoluti

Graph file to TFLite model file

In [ ]:
# Convert exported graph file into TFLite model file
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open(tflite_model_dir+ '/detect.tflite', 'wb') as f:
  f.write(tflite_model)

Dynamic range quantization

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
conv_result = converter.convert()
with open(tflite_model_dir + '/dynrange_quant.tflite', 'wb') as f:
  f.write(conv_result)

Float16 quantization

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
conv_result = converter.convert()
with open(tflite_model_dir + '/float16_quant.tflite', 'wb') as f:
  f.write(conv_result)

Add Model Metadata

In [ ]:
!pip install tflite-support

In [ ]:

!wget -P {project_folder} https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification/metadata/metadata_writer_for_image_classifier.py


--2023-11-21 12:41:09--  https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification/metadata/metadata_writer_for_image_classifier.py
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/tensorflow/examples/blob/master/lite/examples/image_classification/metadata/metadata_writer_for_image_classifier.py [following]
--2023-11-21 12:41:10--  https://github.com/tensorflow/examples/blob/master/lite/examples/image_classification/metadata/metadata_writer_for_image_classifier.py
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 46212 (45K) [text/plain]
Saving to: ‘/content/drive/MyDrive/colab08/metadata_writer_for_image_classifier.py’

metadata_writer_for 100%[===================>]  45.13K  --.-KB/s    in 0.02s   

2023-11-21 12:41:10 (1.93 MB/s) - ‘/content/driv

In [ ]:
from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils

ODWRT = object_detector.MetadataWriter
_MODEL_PATH = tflite_model_dir + '/float16_quant.tflite'
# Task Library expects label files that are in the same format as the one below.
_LABEL_FILE = project_folder + '/labels.txt'
_SAVE_TO_PATH = tflite_model_dir + '/float16_metadata.tflite'
# Normalization parameters is required when reprocessing the image. It is
# optional if the image pixel values are in range of [0, 255] and the input
# tensor is quantized to uint8. See the introduction for normalization and
# quantization parameters below for more details.
# https://www.tensorflow.org/lite/models/convert/metadata#normalization_and_quantization_parameters)
_INPUT_NORM_MEAN = 127.5
_INPUT_NORM_STD = 127.5

# Create the metadata writer.
writer = ODWRT.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), [_INPUT_NORM_MEAN], [_INPUT_NORM_STD],
    [_LABEL_FILE])

# Verify the metadata generated by metadata writer.
print(writer.get_metadata_json())

# Populate the metadata into the model.
writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)

{
  "name": "ObjectDetector",
  "description": "Identify which of a known set of objects might be present and provide information about their positions within the given image or a video stream.",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be detected.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  127.5
                ],
                "std": [
                  127.5
                ]
              }
            }
          ],
          "stats": {
            "max": [
              1.0
            ],
            "min": [
              -1.0
            ]
          }
        }
      ],
     